In [1]:
import time
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime


In [2]:
import pymongo
#print(pymongo)
# Create the client
client = pymongo.MongoClient('localhost', 27017)

# Connect to our database
db = client['deCrime']

# Fetch our series collection
Scrapes = db['Scrapes']

In [3]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [4]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/nick/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [5]:
url = 'http://www.dpdconnect.com/dailybulletin.aspx'
browser.visit(url)

In [6]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

mainTable = soup.find('div', id='gbox_tblDB')
listTable= soup.find('table', id="tblDB")
dateField = browser.find_by_id("mainContent_txtDate2")
dateSubmit = browser.find_by_id("mainContent_lbUpdate")
limitSelector = browser.find_by_css("#pager_center select")
typeSelector = browser.find_by_id("mainContent_ddlType2")
print(typeSelector.value)



AL


In [7]:
def nap(cat=.150):
    time.sleep(cat)


In [8]:
def rebaseHtml():
    nap()
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    mainTable = soup.find('div', id='gbox_tblDB')
    listTable = soup.find('table', id="tblDB")
    dateField = browser.find_by_id("mainContent_txtDate2")
    dateSubmit = browser.find_by_id("mainContent_lbUpdate")
    limitSelector = browser.find_by_css("#pager_center select")
    #limit_selector.select(5)
    limitSelector.select(10000)
    nap()
    limitSelector.click()
    typeSelector = browser.find_by_id("mainContent_ddlType2")
    typeSelector.select("AL")
    nap()


In [9]:
def changeDate(date):
    dateField = browser.find_by_id("mainContent_txtDate2")
    dateSubmit = browser.find_by_id("mainContent_lbUpdate")
    dateNow=dateField.value
    print(f'{dateNow}=>{date}')
    dateField.fill(date)
    dateSubmit.click()
    rebaseHtml()

In [10]:
def scrapeAllIncidents():
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    mainTable = soup.find('div', id='gbox_tblDB')
    listTable = soup.find('table', id="tblDB")
    listIncidents = listTable.find_all('tr')
    print(len(listIncidents))
    print(listIncidents[0])
    for incident in listIncidents:
        caseType = incident['id'].split("<br>")[0].split("  ")[0]        
        caseNum = incident['id'].split("<br>")[0].split("  ")[1]
        tds = incident.find_all('td')
        print(incident.find_all('td')[3]['title'])#Charge

        print(incident.find_all('td')[1]['title'].split("(")[1].split(")")[0])#Race/Sex/Age tuple from longDe
        #print(incident.find_all('td')[4]['title'])#Last
        #print(incident.find_all('td')[5]['title'])#First
        #print(incident.find_all('td')[6]['title'])#Middle
        print(incident.find_all('td')[7]['title'])#Race
        print(incident.find_all('td')[8]['title'])#Sex
        print(incident.find_all('td')[9]['title'])#Desc
        print(incident.find_all('td')[10]['title'])#Address
        Scrapes.insert_one({"caseType": caseType,
                            "caseNum": caseNum,
                            "charge":tds[3]['title'],
                            "rsa": tds[1]['title'].split("(")[1].split(")")[0],
                            "race": tds[7]['title'],
                            "sex": tds[8]['title'],
                            "description": tds[9]['title'],
                            "address": tds[10]['title']
                            })
        
    

In [11]:
changeDate('6/6/2015')

3/29/2021=>6/6/2015


In [12]:
changeDate('12/25/2015')
nap(.5)
scrapeAllIncidents()

6/6/2015=>12/25/2015
27
<tr class="ui-widget-content jqgrow ui-row-ltr" id="AR  15014207&lt;br&gt;&lt;I&gt;1) Domestic Battery (m)...&lt;/I&gt;" role="row"><td role="gridcell" style="width: 180px;" title="AR  150142071) Domestic Battery (m)...">AR  15014207<br/><i>1) Domestic Battery (m)...</i></td><td role="gridcell" style="width: 432px;" title="Bond, Kiedren J (B /M/32) Arrest on chrg of 1) Domestic Battery (M) and 2) Contempt Of Court (F), at 525 W Macon St, Decatur, IL,  on 12/25/2015 09:40.">Bond, Kiedren J (B /M/32) Arrest on chrg of 1) Domestic Battery (M) and 2) Contempt Of Court (F), at 525 W Macon St, Decatur, IL,  on 12/25/2015 09:40.</td><td role="gridcell" style="width: 108px;" title="Sturdivant, L A">Sturdivant, L A</td><td role="gridcell" style="display:none;width: 150px;" title="1) Domestic Battery (M) and 2) Contempt Of Court (F),">1) Domestic Battery (M) and 2) Contempt Of Court (F),</td><td role="gridcell" style="display:none;width: 150px;" title="Bond">Bond</td><td 

In [13]:
#startDate = '10/09/2012' #experimentally determined enough for jazz
startDate = date(2012,10,9)
print (startDate.strftime("%m/%d/%Y"))

10/09/2012


In [14]:
def nextDate(dateOb):
    return dateOb + timedelta(days=1)

In [15]:
threeDaysLater = nextDate(nextDate(nextDate(startDate)))
print (threeDaysLater.strftime("%m/%d/%Y"))

10/12/2012


In [19]:
print(date.today().strftime("%m/%d/%Y"))
endDate= date(2012,10,16)
endDate = date.today()

03/29/2021


In [18]:
currentDate = startDate
while currentDate < endDate:
    nap()
    changeDate(currentDate.strftime("%m/%d/%Y"))
    nap()
    scrapeAllIncidents()
    currentDate=nextDate(currentDate)
    print("woohoo")

=>10/09/2012
58
<tr class="ui-widget-content jqgrow ui-row-ltr" id="AR  12042380&lt;br&gt;&lt;I&gt;Criminal Damage To Prop...&lt;/I&gt;" role="row"><td role="gridcell" style="width: 180px;" title="AR  12042380Criminal Damage To Prop...">AR  12042380<br/><i>Criminal Damage To Prop...</i></td><td role="gridcell" style="width: 432px;" title="Adams, Allen Nathaniel (B /M/42) Arrest on chrg of Criminal Damage To Property, M (M), at 1336 E Orchard St, Decatur, IL,  on 10/9/2012 00:01.">Adams, Allen Nathaniel (B /M/42) Arrest on chrg of Criminal Damage To Property, M (M), at 1336 E Orchard St, Decatur, IL,  on 10/9/2012 00:01.</td><td role="gridcell" style="width: 108px;" title="Cline, T L">Cline, T L</td><td role="gridcell" style="display:none;width: 150px;" title="Criminal Damage To Property, M (M),">Criminal Damage To Property, M (M),</td><td role="gridcell" style="display:none;width: 150px;" title="Adams">Adams</td><td role="gridcell" style="display:none;width: 150px;" title="Allen">Allen

10/09/2012=>10/10/2012
10
<tr class="ui-widget-content jqgrow ui-row-ltr" id="AR  12027795&lt;br&gt;&lt;I&gt;Traf Cancelled/suspende...&lt;/I&gt;" role="row"><td role="gridcell" style="width: 180px;" title="AR  12027795Traf Cancelled/suspende...">AR  12027795<br/><i>Traf Cancelled/suspende...</i></td><td role="gridcell" style="width: 432px;" title="Oneal, Andrew  (W /M/30) Arrest on chrg of Traf Cancelled/suspended/revoked Registration, M (M), at 610 W Sawyer, Decatur, IL,  on 10/10/2012 16:21.">Oneal, Andrew  (W /M/30) Arrest on chrg of Traf Cancelled/suspended/revoked Registration, M (M), at 610 W Sawyer, Decatur, IL,  on 10/10/2012 16:21.</td><td role="gridcell" style="width: 108px;" title="Daniels, K J">Daniels, K J</td><td role="gridcell" style="display:none;width: 150px;" title="Traf Cancelled/suspended/revoked Registration, M (M),">Traf Cancelled/suspended/revoked Registration, M (M),</td><td role="gridcell" style="display:none;width: 150px;" title="Oneal">Oneal</td><td role="gr

10/11/2012=>10/12/2012
74
<tr class="ui-widget-content jqgrow ui-row-ltr" id="AR  12038828&lt;br&gt;&lt;I&gt;Domestic Battery, M (m), &lt;/I&gt;" role="row"><td role="gridcell" style="width: 180px;" title="AR  12038828Domestic Battery, M (m), ">AR  12038828<br/><i>Domestic Battery, M (m), </i></td><td role="gridcell" style="width: 432px;" title="Milan, Javaris J (B /M/20) Arrest on chrg of Domestic Battery, M (M), at 333 S Franklin, Decatur, IL,  on 10/12/2012 17:17.">Milan, Javaris J (B /M/20) Arrest on chrg of Domestic Battery, M (M), at 333 S Franklin, Decatur, IL,  on 10/12/2012 17:17.</td><td role="gridcell" style="width: 108px;" title="Siefman, A K">Siefman, A K</td><td role="gridcell" style="display:none;width: 150px;" title="Domestic Battery, M (M),">Domestic Battery, M (M),</td><td role="gridcell" style="display:none;width: 150px;" title="Milan">Milan</td><td role="gridcell" style="display:none;width: 150px;" title="Javaris">Javaris</td><td role="gridcell" style="display:none;

10/12/2012=>10/13/2012
53
<tr class="ui-widget-content jqgrow ui-row-ltr" id="AR  12046590&lt;br&gt;&lt;I&gt;1) Traf Driving Under T...&lt;/I&gt;" role="row"><td role="gridcell" style="width: 180px;" title="AR  120465901) Traf Driving Under T...">AR  12046590<br/><i>1) Traf Driving Under T...</i></td><td role="gridcell" style="width: 432px;" title="Cunningham, Gregory J (W /M/26) Arrest on chrg of 1) Traf Driving Under The Influence (M), 2) Traf Driving Under The Influence (M), 3) Traf Operating Uninsured Motor Vehicle (M), and 4) Traf All Other Traffic Ivc (M), at 399 W Main/n Edward, Decatur, IL,  on 10/13/2012 01:05.">Cunningham, Gregory J (W /M/26) Arrest on chrg of 1) Traf Driving Under The Influence (M), 2) Traf Driving Under The Influence (M), 3) Traf Operating Uninsured Motor Vehicle (M), and 4) Traf All Other Traffic Ivc (M), at 399 W Main/n Edward, Decatur, IL,  on 10/13/2012 01:05.</td><td role="gridcell" style="width: 108px;" title="Owens, J A">Owens, J A</td><td role="grid

10/13/2012=>10/14/2012
49
<tr class="ui-widget-content jqgrow ui-row-ltr" id="AR  12046090&lt;br&gt;&lt;I&gt;1) Reckless Conduct (m)...&lt;/I&gt;" role="row"><td role="gridcell" style="width: 180px;" title="AR  120460901) Reckless Conduct (m)...">AR  12046090<br/><i>1) Reckless Conduct (m)...</i></td><td role="gridcell" style="width: 432px;" title="Westbrook, Lloyd K (W /M/45) Arrest on chrg of 1) Reckless Conduct (M) and 2) Domestic Battery (M), at 3035 N Water, Decatur, IL,  on 10/14/2012 08:00.">Westbrook, Lloyd K (W /M/45) Arrest on chrg of 1) Reckless Conduct (M) and 2) Domestic Battery (M), at 3035 N Water, Decatur, IL,  on 10/14/2012 08:00.</td><td role="gridcell" style="width: 108px;" title="Miller, C L">Miller, C L</td><td role="gridcell" style="display:none;width: 150px;" title="1) Reckless Conduct (M) and 2) Domestic Battery (M),">1) Reckless Conduct (M) and 2) Domestic Battery (M),</td><td role="gridcell" style="display:none;width: 150px;" title="Westbrook">Westbrook</td><t